<a href="https://colab.research.google.com/github/mrahmedmamdouh/app/blob/startups/train%20and%20export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --upgrade protobuf


In [ ]:
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


In [ ]:
!git clone 'https://github.com/tensorflow/models.git'

In [ ]:
from google.colab import drive
drive.mount('/content/models')


In [ ]:
%cd '/content/models/MyDrive/models'


In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/models/MyDrive/models/research/
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/MyDrive/models/research/:/content/models/MyDrive/models/research/slim'

!python setup.py build
!python setup.py install


In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)


In [ ]:
!ls "/content/models/MyDrive/models/research/object_detection"


In [ ]:
!pip install tf_slim
%cd /content/models/MyDrive/models/research/object_detection
os.environ['PYTHONPATH'] += ':/content/models/MyDrive/models/research/:/content/models/MyDrive/models/research/slim'

!python train.py --train_dir=/content/models/MyDrive/models/research/object_detection/training_data/ --pipeline_config_path=/content/models/MyDrive/models/research/object_detection/training_data/pipeline.config --logtostderr


In [ ]:
#  .ckpt needs to be updated every time to match last .ckpt generated
#  .config needs to be updated when changing model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path /content/models/MyDrive/models/research/object_detection/training_data/pipeline.config --trained_checkpoint_prefix /content/models/MyDrive/models/research/object_detection/training_data/model.ckpt-26058 --output_directory new_graph


In [ ]:
!zip -r model_graph.zip new_graph

In [ ]:
import tensorflow as tf
gf = tf.GraphDef()   
m_file = open('/content/models/MyDrive/models/research/object_detection/new_graph_ssd/tflite_graph.pb','rb')
gf.ParseFromString(m_file.read())
for n in gf.node:
    print( n.name )

In [ ]:
!python export_tflite_ssd_graph.py --pipeline_config_path /content/models/MyDrive/models/research/object_detection/training_data/pipeline.config --trained_checkpoint_prefix /content/models/MyDrive/models/research/object_detection/training_data/model.ckpt-26058 --output_directory new_graph_ssd

In [ ]:
!zip -r model_graph_ssd.zip new_graph_ssd